# inferCNV Automated Terra Script

#### This script generates the input files and all commands needed to run inferCNV on terra per sample

In [ ]:
# setup environment
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import os

In [ ]:
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

### Loading and combining samples from cellranger count filtered count matricies (unprocessed)


In [ ]:
sample1 = sc.read_10x_h5('/your/unprocessed/raw_count_matrices/sample1.h5')
sample2 = sc.read_10x_h5('/your/unprocessed/raw_count_matrices/sample2.h5')
sample3 = sc.read_10x_h5('/your/unprocessed/raw_count_matrices/sample3.h5')
sample4 = sc.read_10x_h5('/your/unprocessed/raw_count_matrices/sample4.h5')

sample1.var_names_make_unique()
sample2.var_names_make_unique()
sample3.var_names_make_unique()
sample4.var_names_make_unique()

In [ ]:
sample1.obs['sample'] = 'sample1'
sample2.obs['sample'] = 'sample2'
sample3.obs['sample'] = 'sample3'
sample4.obs['sample'] = 'sample4'

In [ ]:
adata = sample1.concatenate(sample2, sample3, sample4)

## Load processed/annotated count matrix

In [ ]:
adata2 = sc.read_h5ad('/your/processed/count_matrix.h5ad')

## Generate files for R inferCNV on Terra

In [ ]:
#these are the directories in which all the generated files (formatted count matricies and annotations) and infercnv outputs will be placed
local_infercnv_files = '/your/local/path/inferCNV_inputs'
local_infercnv_outputs = '/your/local/path/inferCNV_outputs'

In [ ]:
# download script to create gene positions file from inferCNV github repo here: 
# https://github.com/broadinstitute/infercnv/blob/master/scripts/gtf_to_position_file.py
# then run:
ref_genome_file = 'your-reference-genome-gtf-file-here.gtf'
print('python gtf_to_position_file.py %s %s/gene_ordering.txt --attribute_name=gene_name' % (ref_genome_file, local_infercnv_files))

In [ ]:
#set your terra workspace and google bucket/folders
user_email = 'your-email@broadinstitute.org'
terra_workspace = "namespace/workspace"

gs_bucket = "gs://fc-your-bucket-here"
gs_inputs_folder = gs_bucket + "/inferCNV_inputs"
gs_outputs_folder = gs_bucket + "/inferCNV_outputs"

In [ ]:
#filter raw anndata to only include the cells in your processed anndata object and
#add cell type annotations from your processed anndata object to the raw anndata object
adata = adata[adata.obs.index.isin(adata2.obs.index)]
adata.obs = adata.obs.merge(adata2.obs[['broad_clusters']], left_index=True, right_index=True)

## Per Sample (one object/run per sample - recommended option)

In [ ]:
#for every sample generate your formatted raw count matrix and annotations file
#generates one script to automatically run all samples on Terra
#note: infercnv requires every group (cell type in this case) to have at least 2 cells, so any cell types
#with 0 or 1 cells are filtered out

print('Run command below to upload all inferCNV inputs and run on terra:')
print('bash %s/run_terra.sh' % (local_infercnv_files))
terra_script = open("%s/run_terra.sh" % (local_infercnv_files), "w")
terra_script.write('#running inferCNV on all samples on Terra \n')
#uncomment if running on UGER
# terra_script.write('source /broad/software/scripts/useuse\n')
# terra_script.write('reuse Anaconda3\n')
# terra_script.write('use Google-Cloud-SDK\n')
# terra_script.write('gcloud auth login %s\n' % (user_email))
# terra_script.write('source activate /seq/regev_genome_portal/conda_env/cumulus\n')
terra_script.write('#upload all inferCNV inputs to google bucket \n')
terra_script.write('gsutil -m rsync -r %s %s\n' % (local_infercnv_files, gs_inputs_folder))
terra_script.write('#start terra workflows \n')
for sample in adata.obs['sample'].unique().tolist():
    #get base path for sample and make directory
    sample_path = '%s/%s' % (local_infercnv_files, sample)
    if not os.path.exists(sample_path):
        os.makedirs(sample_path)
        
    #subset anndata per sample
    temp = adata[adata.obs["sample"] == sample]
    
    #remove cell types with less than 2 cells
    for celltype in temp.obs.broad_clusters.unique().tolist():
        celltypecount = len(temp.obs.loc[temp.obs.broad_clusters == celltype])
        if celltypecount < 2:
            temp = temp[temp.obs.broad_clusters != celltype]
    
    #get reference list
    ref_list = temp.obs.broad_clusters.unique().tolist()
    ref_list.remove('Tumor')

    #generate annotations file
    annotations_file_path = sample_path + '/' + sample + '_annotations.txt'
    annotations = temp.obs['broad_clusters']
    annotations.to_csv(annotations_file_path, sep='\t', header=False, index=True)
    
    #generate raw count matrix
    count_matrix_file_path = sample_path + '/' + sample + '.counts.matrix'
    count_matrix = pd.DataFrame.sparse.from_spmatrix(temp.X.transpose(), index=temp.var.index, columns=temp.obs.index)
    count_matrix.to_csv(count_matrix_file_path, sep='\t', chunksize=1000000)

    terra_input_file = "%s/%s_input_terra.json" % (sample_path, sample) 

    with open(terra_input_file, "w") as f:
        f.write("{\n")
        f.write("\t\"infercnv.output_directory\" : \"%s/%s\",\n" % (gs_outputs_folder, sample))
        f.write("\t\"infercnv.annotations_file\" : \"%s/%s/%s_annotations.txt\",\n" % (gs_inputs_folder, sample, sample))
        f.write("\t\"infercnv.gene_order_file\" : \"%s/gene_ordering.txt\",\n" % (gs_inputs_folder))
        f.write("\t\"infercnv.raw_counts_matrix\" : \"%s/%s/%s.counts.matrix\",\n" % (gs_inputs_folder, sample, sample))
        f.write("\t\"infercnv.ref_group_names\" : \"\'%s\'\",\n" % (",".join(ref_list)))
        f.write("\t\"infercnv.denoise\" : true,\n")
        f.write("\t\"infercnv.cluster_by_groups\" : false,\n")
        f.write("\t\"infercnv.HMM\" : true,\n")
        f.write("\t\"infercnv.HMM_type\" : \"i6\",\n")
        f.write("\t\"infercnv.cutoff\" : \"0.1\",\n")
        f.write("\t\"infercnv.analysis_mode\" : \"subclusters\",\n")
        f.write("\t\"infercnv.tumor_subcluster_pval\" : \"0.1\",\n")
        f.write("\t\"infercnv.median_filter\" : true,\n")
        #additional arguments are documented here: https://github.com/broadinstitute/infercnv/blob/master/scripts/inferCNV.R
        f.write("\t\"infercnv.additional_args\" : \"--k_obs_groups 10\",\n" )
        f.write("\t\"infercnv.cpu\" : 16,\n")
        f.write("\t\"infercnv.memory\" : \"128GB\"\n")
        f.write("}\n")
        
    terra_script.write("alto terra run -m mparikh/infercnv_caching/9 -i %s/%s/%s_input_terra.json -w '%s'\n" % (local_infercnv_files, sample, sample, terra_workspace))
terra_script.close()

In [ ]:
#these are the outputs you want to download per sample
outputs = [
    'infercnv.preliminary.png',
    'infercnv.png',
    'infercnv.references.txt',
    'infercnv.observations.txt',
    'infercnv.observation_groupings.txt',
    'infercnv.observations_dendrogram.txt'
]

In [ ]:
#generate commands to download outputs for each sample
print('Run command below to download all infercnv outputs:')
print('bash %s/download.sh' % (local_infercnv_outputs))
download_script = open("%s/download.sh" % (local_infercnv_outputs), "w")
download_script.write('#download all inferCNV output files from google bucket\n')
#uncomment if running on UGER
# download_script.write('source /broad/software/scripts/useuse\n')
# download_script.write('use Google-Cloud-SDK\n')
for sample in adata.obs['sample'].unique().tolist():
    #get base path for sample and make directory
    sample_path = '%s/%s' % (local_infercnv_outputs, sample)
    if not os.path.exists(sample_path):
        os.makedirs(sample_path)
    gs_sample_output_folder = gs_outputs_folder + '/' + sample + '/'
    filelist = [gs_sample_output_folder + output for output in outputs]
    textfile = open("%s/%s/filelist.txt" % (local_infercnv_outputs, sample), "w")
    for element in filelist:
        textfile.write(element + "\n")
    textfile.close()
    download_script.write('#%s\n' % (sample))
    download_script.write('cat %s/%s/filelist.txt | gsutil -m cp -I %s/%s/\n' % (local_infercnv_outputs, sample, local_infercnv_outputs, sample))
download_script.close()

## All Samples (one Terra run with full dataset - **WARNING** can take days and be very expensive)

In [ ]:
#generate your formatted raw count matrix and annotations file
#generates one script to automatically run full dataset on Terra
#note: infercnv requires every group (cell type in this case) to have at least 2 cells, so any cell types
#with 0 or 1 cells are filtered out

print('Run command below to upload all inferCNV inputs and run on terra:')
print('bash %s/run_all_terra.sh' % (local_infercnv_files))
terra_script = open("%s/run_all_terra.sh" % (local_infercnv_files), "w")
terra_script.write('#running inferCNV on Terra \n')
#uncomment if running on UGER
# terra_script.write('source /broad/software/scripts/useuse\n')
# terra_script.write('reuse Anaconda3\n')
# terra_script.write('use Google-Cloud-SDK\n')
# terra_script.write('gcloud auth login %s\n' % (user_email))
# terra_script.write('source activate /seq/regev_genome_portal/conda_env/cumulus\n')
terra_script.write('#upload all inferCNV inputs to google bucket \n')
terra_script.write('gsutil -m rsync -r %s %s\n' % (local_infercnv_files, gs_inputs_folder))
terra_script.write('#start terra workflows \n')
#get path and make directory
sample = 'all'
all_path = '%s/%s' % (local_infercnv_files, sample)
if not os.path.exists(all_path):
    os.makedirs(all_path)

#remove cell types with less than 2 cells
for celltype in adata.obs.broad_clusters.unique().tolist():
    celltypecount = len(adata.obs.loc[adata.obs.broad_clusters == celltype])
    if celltypecount < 2:
        adata = adata[adata.obs.broad_clusters != celltype]

#get reference list
ref_list = adata.obs.broad_clusters.unique().tolist()
ref_list.remove('Tumor')

#generate annotations file
annotations_file_path = all_path + '/' + sample + '_annotations.txt'
annotations = adata.obs['broad_clusters']
annotations.to_csv(annotations_file_path, sep='\t', header=False, index=True)

#generate raw count matrix
count_matrix_file_path = all_path + '/' + sample + '.counts.matrix'
count_matrix = pd.DataFrame.sparse.from_spmatrix(adata.X.transpose(), index=adata.var.index, columns=adata.obs.index)
count_matrix.to_csv(count_matrix_file_path, sep='\t', chunksize=1000000)

terra_input_file = "%s/%s_input_terra.json" % (all_path, sample) 

with open(terra_input_file, "w") as f:
    f.write("{\n")
    f.write("\t\"infercnv.output_directory\" : \"%s/%s\",\n" % (gs_outputs_folder, sample))
    f.write("\t\"infercnv.annotations_file\" : \"%s/%s/%s_annotations.txt\",\n" % (gs_inputs_folder, sample, sample))
    f.write("\t\"infercnv.gene_order_file\" : \"%s/gene_ordering.txt\",\n" % (gs_inputs_folder))
    f.write("\t\"infercnv.raw_counts_matrix\" : \"%s/%s/%s.counts.matrix\",\n" % (gs_inputs_folder, sample, sample))
    f.write("\t\"infercnv.ref_group_names\" : \"\'%s\'\",\n" % (",".join(ref_list)))
    f.write("\t\"infercnv.denoise\" : true,\n")
    f.write("\t\"infercnv.cluster_by_groups\" : false,\n")
    f.write("\t\"infercnv.HMM\" : true,\n")
    f.write("\t\"infercnv.HMM_type\" : \"i6\",\n")
    f.write("\t\"infercnv.cutoff\" : \"0.1\",\n")
    f.write("\t\"infercnv.analysis_mode\" : \"subclusters\",\n")
    f.write("\t\"infercnv.tumor_subcluster_pval\" : \"0.1\",\n")
    f.write("\t\"infercnv.median_filter\" : true,\n")
    #additional arguments are documented here: https://github.com/broadinstitute/infercnv/blob/master/scripts/inferCNV.R
    f.write("\t\"infercnv.additional_args\" : \"--k_obs_groups 10\",\n" )
    f.write("\t\"infercnv.cpu\" : 16,\n")
    f.write("\t\"infercnv.memory\" : \"128GB\"\n")
    f.write("}\n")

terra_script.write("alto terra run -m mparikh/infercnv_caching/9 -i %s/%s/%s_input_terra.json -w '%s'\n" % (local_infercnv_files, sample, sample, terra_workspace))
terra_script.close()

In [ ]:
#these are the outputs you want to download per sample
outputs = [
    'infercnv.preliminary.png',
    'infercnv.png',
    'infercnv.references.txt',
    'infercnv.observations.txt',
    'infercnv.observation_groupings.txt',
    'infercnv.observations_dendrogram.txt'
]

In [ ]:
#generate commands to download outputs for each sample
print('Run command below to download infercnv outputs:')
print('bash %s/download_all.sh' % (local_infercnv_outputs))
download_script = open("%s/download_all.sh" % (local_infercnv_outputs), "w")
download_script.write('#download all inferCNV output files from google bucket\n')
#uncomment if running on UGER
# download_script.write('source /broad/software/scripts/useuse\n')
# download_script.write('use Google-Cloud-SDK\n')
sample = 'all'
#get base path for sample and make directory
sample_path = '%s/%s' % (local_infercnv_outputs, sample)
if not os.path.exists(sample_path):
    os.makedirs(sample_path)
gs_sample_output_folder = gs_outputs_folder + '/' + sample + '/'
filelist = [gs_sample_output_folder + output for output in outputs]
textfile = open("%s/%s/filelist.txt" % (local_infercnv_outputs, sample), "w")
for element in filelist:
    textfile.write(element + "\n")
textfile.close()
download_script.write('#%s\n' % (sample))
download_script.write('cat %s/%s/filelist.txt | gsutil -m cp -I %s/%s/\n' % (local_infercnv_outputs, sample, local_infercnv_outputs, sample))
download_script.close()